## Magnitude  and shape distributions for DR2 targets

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import corner as cn
import desitarget.cuts

from astropy.io import fits
from astropy.table import Table, join
from sklearn.mixture import GaussianMixture
from desitarget.targetmask import desi_mask, bgs_mask

In [3]:
class GaussianMixtureModel(object):

    def __init__(self, weights, means, covars, covtype):
        self.weights = weights
        self.means = means
        self.covars = covars
        self.covtype = covtype
        self.n_components, self.n_dimensions = self.means.shape
    
    @staticmethod
    def save(model, filename):
        hdus = fits.HDUList()
        hdr = fits.Header()
        hdr['covtype'] = model.covariance_type
        hdus.append(fits.ImageHDU(model.weights_, name='weights', header=hdr))
        hdus.append(fits.ImageHDU(model.means_, name='means'))
        hdus.append(fits.ImageHDU(model.covariances_, name='covars'))
        hdus.writeto(filename, overwrite=True)

    @staticmethod
    def load(filename):
        hdus = fits.open(filename, memmap=False)
        hdr = hdus[0].header
        covtype = hdr['covtype']
        model = GaussianMixtureModel(
            hdus['weights'].data, hdus['means'].data, hdus['covars'].data, covtype)
        hdus.close()
        return model
        
    def sample(self, n_samples=1, random_state=None):
        
        if self.covtype != 'full':
            return NotImplementedError(
                'covariance type "{0}" not implemented yet.'.format(self.covtype))
        
        # Code adapted from sklearn's GMM.sample()
        if random_state is None:
            random_state = np.random.RandomState()

        weight_cdf = np.cumsum(self.weights)
        X = np.empty((n_samples, self.n_dimensions))
        rand = random_state.rand(n_samples)
        # decide which component to use for each sample
        comps = weight_cdf.searchsorted(rand)
        # for each component, generate all needed samples
        for comp in range(self.n_components):
            # occurrences of current component in X
            comp_in_X = (comp == comps)
            # number of those occurrences
            num_comp_in_X = comp_in_X.sum()
            if num_comp_in_X > 0:
                X[comp_in_X] = random_state.multivariate_normal(
                    self.means[comp], self.covars[comp], num_comp_in_X)
        return X

In [33]:
def keep(data):
    #takes in table data for each object type and returns a numpy array of DECAM/WISE magnitudes and shapes
    gflux = data['decam_flux'][:,1]
    rflux = data['decam_flux'][:,2]
    zflux = data['decam_flux'][:,4]
    w1flux = data['wise_flux'][:,0]
    w2flux = data['wise_flux'][:,1]
    w3flux = data['wise_flux'][:,2]
    w4flux = data['wise_flux'][:,3]
    
    #only keep non-zero, non-negative flux values to convert to magnitudes
    keep = (gflux > 0) & (rflux > 0) & (zflux > 0) & (w1flux > 0) & (w2flux > 0) & (w3flux > 0) & (w4flux >0)
    
    gg = 22.5-2.5*np.log10(gflux[keep])
    rr = 22.5-2.5*np.log10(rflux[keep])
    zz = 22.5-2.5*np.log10(zflux[keep])
    w1 = 22.5-2.5*np.log10(w1flux[keep])
    w2 = 22.5-2.5*np.log10(w2flux[keep])
    w3 = 22.5-2.5*np.log10(w3flux[keep])
    w4 = 22.5-2.5*np.log10(w4flux[keep])
    
    exp_r = data['shapeExp_r'][keep]
    exp_e1 = data['shapeExp_e1'][keep]
    exp_e2 = data['shapeExp_e2'][keep]
    dev_r = data['shapeDev_r'][keep]
    dev_e1 = data['shapeDev_e1'][keep]
    dev_e2 = data['shapeDev_e2'][keep]
    fracdev = data['fracDev'][keep]
    
    return np.array([gg, rr, zz, w1, w2, w3, w4, exp_r, exp_e1, exp_e2, dev_r, dev_e1, dev_e2, fracdev]).T


def get_bic(data, components_range):
    bic = []
    #generate bic for each component in the range given
    for comp in components_range:
        model = GaussianMixture(n_components=comp, covariance_type='full')
        model.fit(data)
        bic.append(model.bic(data))
        #print('Component: {:d}'.format(comp))
    return bic

def plot_bic(bic, components_range):
    fig, ax = plt.subplots(1, 1, figsize=(8,4))
    ax.plot(components_range, np.asarray(np.asarray(bic)/100), marker='s', ls='-')
    ax.set_xlabel('Number of Gaussian Components')
    ax.set_ylabel('Bayesian Information Criterion/100')
    plt.title('Optimal number of components = {:d}'.format(np.argmin(bic)))
    plt.show()  


def make_gmm_model(X_data, components_range, model_filename=None, seed=123, bic_plot=False):
    #list of bic values for given range of components
    bic = get_bic(X_data, components_range)
    #option to plot bic values
    if bic_plot:
        plot_bic(bic, components_range)
    #index of lowest bic value gives the optimal number of components
    n_comp = np.argmin(bic)
    gen = np.random.RandomState(seed)
    model = GaussianMixture(n_components=n_comp, covariance_type="full", random_state=gen).fit(X_data)
    if model_filename:
        GaussianMixtureModel.save(model, model_filename)
        print('Saved GMM as {:s}.'.format(model_filename))
    else:
        return model

In [5]:
def data_cross_split(params):
    #splits data into training and validation sets
    tot = len(params)
    half = np.floor(tot/2.).astype(int)
    data = params[:half]
    cross = params[half:tot]
    return data, cross

def sample(target_type, n_targets, random_state=None):
    if target_type == None:
        model = GaussianMixtureModel.load(filename)
    if target_type == 'LRG':
        model = GaussianMixtureModel.load('data/v2/lrg_gmm.fits')
    elif target_type == 'ELG':
        model = GaussianMixtureModel.load('data/v2/elg_gmm.fits')
    elif target_type == 'QSO':
        model = GaussianMixtureModel.load('data/v2/qso_gmm.fits')
    elif target_type == 'BGS':
        model = GaussianMixtureModel.load('data/v2/bgs_gmm.fits')
    return  model.sample(n_targets, random_state)

In [6]:
#import selection dr2 targets that have passed selection cuts
lrg = Table.read('data/v2/lrg_cuts.fits', format='fits')
elg = Table.read('data/v2/elg_cuts.fits', format='fits')
qso = Table.read('data/v2/qso_cuts.fits', format='fits')
bgs = Table.read('data/v2/bgs_cuts.fits', format='fits')

In [23]:
#rng for sampling
seed = 123
gen = np.random.RandomState(seed)

#number of components to test for bic
components_range = range(1,36)

In [ ]:
lrg_params = keep(lrg)
#make sure data, sample and cross-validation sets are of same size for comparison
lrg_data, lrg_cross = data_cross_split(lrg_params)

make_gmm_model(lrg_data, components_range, model_filename='data/v2/lrg_gmm.fits', bic_plot=True)

lrg_sample = sample('LRG', n_targets=len(lrg_data), random_state=gen)